In [ ]:
import sys
sys.path.append('../src/sunscan')

import os
import logging
import datetime
from pathlib import Path
from zipfile import ZipFile
from tempfile import TemporaryDirectory
import geopandas as gpd

from definitions import BoundingBox
from sunscan_process import predict_house_roofs
from solar_energy import calculate_solar_energy

logger = logging.getLogger()
logging.basicConfig(
    level=os.environ.get('LOGLEVEL', 'INFO').upper()
)

MODELS_BASE_PATH = Path('../models')
OUTPUTS_BASE_PATH = Path('../data/outputs')
OUTPUTS_BASE_PATH.mkdir(parents=True, exist_ok=True)

DEFAULT_MODEL_NAME = 'sam_vit_h_4b8939.pth'

payload = {
    'bounding_box': [-73.2316, 9.587, -73.2309, 9.5875],
    'panel_size': 4,
    'available_area': 0.5,
    'panel_power': 400,
    'dimension_factor': 0.8
}

start = datetime.datetime(2024, 1, 1)
end = datetime.datetime(2024, 12, 31)

In [ ]:
def zip_shp_result(gdf: gpd.GeoDataFrame, zip_path: Path):
    with TemporaryDirectory() as tempdir:
        temp_path = Path(tempdir) / 'solar_energy'
        gdf.to_file(temp_path)

        with ZipFile(zip_path.as_posix(), mode='w') as archive:
            for file_path in temp_path.iterdir(): 
                archive.write(file_path.as_posix(), file_path.name)

In [ ]:
model_checkpoint = MODELS_BASE_PATH / DEFAULT_MODEL_NAME

with TemporaryDirectory() as tempdir:
    house_roofs_shp_path = predict_house_roofs(
        BoundingBox(*payload['bounding_box']),
        OUTPUTS_BASE_PATH,
        Path(tempdir),
        model_checkpoint
    )

    start = datetime.datetime(2024, 1, 1)
    end = datetime.datetime(2024, 12, 31)

    solar_energy = calculate_solar_energy(
        house_roofs_shp_path,
        start,
        end,
        payload['panel_size'],
        payload['available_area'],
        payload['panel_power'] / 1000,
        payload['dimension_factor']
    )

    zip_shp_result(solar_energy, OUTPUTS_BASE_PATH / 'solar_energy_shp.zip')
    solar_energy.to_excel(OUTPUTS_BASE_PATH / 'solar_energy.xlsx')